In [33]:
# Imports and Configuration  
import glob  
import os  
import pandas as pd  
import sqlite3 
import numpy as np
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np

In [34]:
# Paths and patterns  
data_root     = "data/output_by_college_clean"  
indiv_pattern = os.path.join(data_root, "*", "*", "individual_stats_overall.csv")  
team_pattern  = os.path.join(data_root, "*", "*", "team_stats.csv")  

In [35]:
directory = Path("data/output_by_college" )
csv_count = len(list(directory.rglob('*.csv')))
print(f"Number of CSV files: {csv_count}")


Number of CSV files: 152


In [36]:
# College rename map  
def get_college_map():  
    return {  
        'gojaspers_com':           "manhattan",  
        'mountathletics_com':      "mount st marys",  
        'purpleeagles_com':        "niagara",  
        'sacredheartpioneers_com': "sacred heart",  
        'gobobcats_com':           "quinnipiac",  
        'merrimackathletics_com':  "merrimack",  
        'goredfoxes_com':          "marist",  
        'fairfieldstags_com':      "fairfield",  
        'ionagaels_com':           "iona",  
        'sienasaints_com':         "siena",  
        'gogriffs_com':            "canisius",  
        'saintpeterspeacocks_com': "saint peters",  
        'gobroncs_com':            "rider",  
    }  

college_map = get_college_map()

In [37]:
# Slugify helper  
def slugify(col: str) -> str:  
    """Convert column names to lowercase underscores and replace % and spaces."""  
    s = col.strip().lower()  
    s = s.replace('%', '_pct').replace('/', '_per_')  
    for ch in [' ', '-', ':', '(', ')']:  
        s = s.replace(ch, '_')  
    while '__' in s:  
        s = s.replace('__', '_')  
    return s


In [38]:
# Build master_individual_stats.csv  
indiv_paths = glob.glob(indiv_pattern)  
indiv_dfs   = []  

for path in indiv_paths:  
    parts   = path.split(os.sep)  
    college = parts[-3]  
    season  = parts[-2]  

    df = pd.read_csv(path)  
    df.columns = [slugify(c) for c in df.columns]  
    df['college'] = college  
    df['season']  = season  
    indiv_dfs.append(df)  

df_indiv = pd.concat(indiv_dfs, ignore_index=True)  
df_indiv['college'] = df_indiv['college'].replace(college_map)  
df_indiv.to_csv('master_individual_stats.csv', index=False)  
print('Saved master_individual_stats.csv', df_indiv.shape)
print(df_indiv.head())


Saved master_individual_stats.csv (2065, 28)
   player_number           player  gp    gs  minutes_tot  minutes_avg  fg_fgm  \
0             35     Kayla Grimme  31  30.0          951         30.7     179   
1             15      Amani Tatum  31  31.0         1041         33.6     133   
2              2      Gabby Cajou  31   1.0          915         29.5     100   
3             30     Mikki Guiton  30  11.0          478         15.9      62   
4             14  Taylor Williams  30  23.0          664         22.1      42   

   fg_fga  fg_fg_pct  3pt  ...  rebounds_def  rebounds_tot  rebounds_avg  pf  \
0     420      0.426   26  ...           153           263           8.5  77   
1     334      0.398   71  ...            99           125           4.0  75   
2     210      0.476   16  ...            52            71           2.3  71   
3     150      0.413    1  ...            43            69           2.3  58   
4     145      0.290   26  ...            69            89          

In [39]:
# Build master_team_stats.csv  
team_paths = glob.glob(team_pattern)  
records    = []  

for path in team_paths:  
    parts   = path.split(os.sep)  
    college = parts[-3]  
    season  = parts[-2]  

    df    = pd.read_csv(path)  
    total = df.iloc[0]  

    rec = {'college': college, 'season': season}  
    for raw_col, val in total.items():  
        if pd.isna(raw_col) or str(raw_col).strip() == '':  
            continue  
        key = slugify(raw_col)  
        rec[key] = val  
    records.append(rec)  

df_team = pd.DataFrame(records)  
df_team['college'] = df_team['college'].replace(college_map)  

cols = ['college', 'season'] + [c for c in df_team.columns if c not in ('college','season')]  
df_team = df_team[cols]  
df_team.to_csv('master_team_stats.csv', index=False)  
print('Saved master_team_stats.csv', df_team.shape)
print(df_team.head())


Saved master_team_stats.csv (152, 28)
     college   season  player_number player  gp  gs  minutes_tot  minutes_avg  \
0  manhattan  2017-18            NaN  Total  31 NaN         6275        202.4   
1  manhattan  2016-17            NaN  Total  30 NaN         6000        200.0   
2  manhattan  2019-20            NaN  Total  29 NaN         5850        201.7   
3  manhattan  2021-22            NaN  Total  32 NaN         6400        200.0   
4  manhattan  2018-19            NaN  Total  31 NaN         6225        200.8   

   fg_fgm  fg_fga  ...  scoring_avg  rebounds_off  rebounds_def  rebounds_tot  \
0     687    1746  ...        58.65           375           712          1087   
1     615    1609  ...        54.93           297           681           978   
2     683    1788  ...        61.21           365           721          1086   
3     791    1972  ...        66.19           475           922          1397   
4     644    1802  ...        54.10           411           759       

In [40]:
# Load CSVs into SQLite  
DB_PATH = 'wbb_stats.db'  
conn    = sqlite3.connect(DB_PATH)  

pd.read_csv('master_individual_stats.csv').to_sql('individual_stats', conn, if_exists='replace', index=False)  

pd.read_csv('master_team_stats.csv').to_sql('team_stats', conn, if_exists='replace', index=False)  

with conn:  
    conn.execute('CREATE INDEX IF NOT EXISTS idx_indiv ON individual_stats(college, season)')  
    conn.execute('CREATE INDEX IF NOT EXISTS idx_team  ON team_stats     (college, season)')  

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)  
print('Tables in DB:', tables['name'].tolist())  
conn.close()


Tables in DB: ['individual_stats', 'team_stats']


In [41]:
# Load from SQLite
conn = sqlite3.connect('wbb_stats.db')
df_i = pd.read_sql('SELECT * FROM individual_stats', conn)
df_t = pd.read_sql('SELECT * FROM team_stats',        conn)
conn.close()

# Merge on college & season; individual/team suffixes
df = df_i.merge(df_t, on=['college','season'], how='left', suffixes=('_ind','_team'))

# Helper to pick the correct column name (unchanged)
def pick(base_names, suf):
    for base in base_names:
        name = f"{base}{suf}"
        if name in df.columns:
            return name
    raise KeyError(f"No column found among {[b+suf for b in base_names]}")

# Identify key columns (with suffix) — your existing picks
pn_col   = pick(['player_number'], '_ind')
p_col    = pick(['player'],        '_ind')
gp_col   = pick(['gp'],            '_ind')
gs_col   = pick(['gs'],            '_ind')
m_ind    = pick(['minutes_tot','minutes'], '_ind')
m_team   = pick(['minutes_tot','minutes'], '_team')

pts_ind  = pick(['scoring_pts','pts'],         '_ind')
reb_tot  = pick(['rebounds_tot','reb'],         '_ind')
ast_ind  = pick(['ast'],                       '_ind')
stl_ind  = pick(['stl'],                       '_ind')
blk_ind  = pick(['blk'],                       '_ind')
to_ind   = pick(['to','tov'],                  '_ind')

fgm_ind  = pick(['fg_fgm','fgm'],              '_ind')
fga_ind  = pick(['fg_fga','fga'],              '_ind')
three_ind= pick(['3pt'],                       '_ind')
fta_ind  = pick(['ft_fta','fta'],              '_ind')

fga_team = pick(['fg_fga','fga'],              '_team')
fta_team = pick(['ft_fta','fta'],              '_team')
to_team  = pick(['to','tov'],                  '_team')

# Compute per-40 (unchanged)
df['pts_per40'] = np.where(df[m_ind]>0, df[pts_ind]/df[m_ind]*40, 0)
df['reb_per40'] = np.where(df[m_ind]>0, df[reb_tot]/df[m_ind]*40, 0)
df['ast_per40'] = np.where(df[m_ind]>0, df[ast_ind]/df[m_ind]*40, 0)
df['stl_per40'] = np.where(df[m_ind]>0, df[stl_ind]/df[m_ind]*40, 0)
df['blk_per40'] = np.where(df[m_ind]>0, df[blk_ind]/df[m_ind]*40, 0)
df['tov_per40'] = np.where(df[m_ind]>0, df[to_ind]/df[m_ind]*40, 0)

# Effective FG%
df['eFG_pct'] = np.where(df[fga_ind]>0,
    (df[fgm_ind] + 0.5*df[three_ind]) / df[fga_ind], 0)

# True Shooting%
df['TS_pct'] = np.where((df[fga_ind] + 0.44*df[fta_ind])>0,
    df[pts_ind] / (2*(df[fga_ind] + 0.44*df[fta_ind])), 0)

# Usage Rate%
u_num = df[fga_ind] + 0.44*df[fta_ind] + df[to_ind]
u_den = df[fga_team] + 0.44*df[fta_team] + df[to_team]
df['USG_pct'] = np.where((df[m_ind]>0)&(u_den>0),
    100 * (u_num * df[m_team])/(df[m_ind]*u_den), 0)

# Rebound Rates
df['ORB_pct'] = np.where(
    (df['rebounds_off_ind']+df['rebounds_def_team'])>0,
    100*df['rebounds_off_ind']/(df['rebounds_off_ind']+df['rebounds_def_team']),0)
df['DRB_pct'] = np.where(
    (df['rebounds_def_ind']+df['rebounds_off_team'])>0,
    100*df['rebounds_def_ind']/(df['rebounds_def_ind']+df['rebounds_off_team']),0)

# Assist-to-Turnover & Assist Rate
df['AST_per_TO'] = np.where(df[to_ind]>0, df[ast_ind]/df[to_ind], np.nan)
df['AST_pct']    = np.where((df[ast_ind]+df[to_ind])>0,
    100*df[ast_ind]/(df[ast_ind]+df[to_ind]), 0)

# Enrichement Block
# 2-pt / 3-pt breakdown
df['fga_2pt'] = df[fga_ind] - df[three_ind]
df['fgm_2pt'] = df[fgm_ind] - df[three_ind]
df['2pt_pct'] = np.where(df['fga_2pt']>0,
    df['fgm_2pt'] / df['fga_2pt'],
    np.nan)

# Free-throw rate
df['FTr'] = np.where(df[fga_ind]>0,
    df[fta_ind] / df[fga_ind],
    0)

# Points per Field-Goal Attempt
df['PPP'] = np.where(df[fga_ind]>0,
    df[pts_ind] / df[fga_ind],
    np.nan)

# Estimate possessions
df['possessions'] = (
    df[fga_ind]
  + 0.44 * df[fta_ind]
  + df[to_ind]
)
df['possessions'].replace(0, np.nan, inplace=True)

# Per-100 possession rates
for stat in ['scoring_pts_ind','ast_ind','rebounds_tot_ind','stl_ind','blk_ind','to_ind']:
    name = stat.replace('_ind','') + '_per100'
    df[name] = df[stat] / df['possessions'] * 100




/var/folders/1p/z7sxpqkn40d8zz3wx8qqp0980000gn/T/ipykernel_4590/1439756971.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['possessions'].replace(0, np.nan, inplace=True)


In [42]:
# Per‐Game Averages
# Compute basic per‐game numbers to capture raw volume vs. rate trade‐offs
df['mins_per_game'] = df[m_ind]    / df[gp_col]
df['pts_per_game']  = df[pts_ind]  / df[gp_col]
df['ast_per_game']  = df[ast_ind]  / df[gp_col]
df['reb_per_game']  = df[reb_tot]  / df[gp_col]
df['stl_per_game']  = df[stl_ind]  / df[gp_col]
df['blk_per_game']  = df[blk_ind]  / df[gp_col]
df['to_per_game']   = df[to_ind]   / df[gp_col]


In [43]:
# Share Metrics
# What fraction of team production each player accounts for
for stat in ['scoring_pts','ast','rebounds_tot','stl','blk','to']:
    ind  = f"{stat}_ind"
    team = f"{stat}_team"
    df[f"{stat}_share"] = np.where(
        df[team] > 0,
        df[ind] / df[team],
        0
    )


In [44]:
team_columns = [col for col in df.columns if "_team" in col]
print("Columns with '_team' in the name:", team_columns)


Columns with '_team' in the name: ['player_number_team', 'player_team', 'gp_team', 'gs_team', 'minutes_tot_team', 'minutes_avg_team', 'fg_fgm_team', 'fg_fga_team', 'fg_fg_pct_team', '3pt_team', '3pt_3pta_team', '3pt_3pt_pct_team', 'ft_ftm_team', 'ft_fta_team', 'ft_ft_pct_team', 'scoring_pts_team', 'scoring_avg_team', 'rebounds_off_team', 'rebounds_def_team', 'rebounds_tot_team', 'rebounds_avg_team', 'pf_team', 'ast_team', 'to_team', 'stl_team', 'blk_team']


In [45]:
# Efficiency Differential
# Compute True Shooting Differential: how a player’s TS% compares to their team’s TS%

# Identify the correct team‐level columns using your pick() helper
pts_team = pick(['scoring_pts','pts'], '_team')
fga_team = pick(['fg_fga','fga'],        '_team')
fta_team = pick(['ft_fta','fta'],        '_team')

# Compute the team’s True Shooting %
df['team_TS_pct'] = np.where(
    (df[fga_team] + 0.44 * df[fta_team]) > 0,
    df[pts_team] / (2 * (df[fga_team] + 0.44 * df[fta_team])),
    0
)

# Subtract to get the differential
df['TS_diff'] = df['TS_pct'] - df['team_TS_pct']


In [46]:
# Assist & Turnover Ratios
# Assists per made field‐goal; turnover rate per opportunity
df['ast_per_fgm'] = np.where(
    df[fgm_ind] > 0,
    df[ast_ind] / df[fgm_ind],
    0
)
df['tov_rate'] = np.where(
    (df[fga_ind] + 0.44*df[fta_ind] + df[to_ind]) > 0,
    df[to_ind] / (df[fga_ind] + 0.44*df[fta_ind] + df[to_ind]),
    0
)


In [47]:
# Hollinger’s Game Score
# Single‐number box‐score impact metric
# (requires you have ft_ftm_ind or adjust if named differently)
df['game_score'] = (
     df[pts_ind]
   + 0.4 * df[fgm_ind]
   - 0.7 * df[fga_ind]
   - 0.4 * (df[fta_ind] - df['ft_ftm_ind'])
   + 0.7 * df[reb_tot]
   + 0.7 * df[ast_ind]
   + 0.7 * df[stl_ind]
   + 0.7 * df[blk_ind]
   - 0.4 * df['pf_ind']     # if you have personal fouls
   - df[to_ind]
)
df['game_score_per40'] = np.where(
    df[m_ind] > 0,
    df['game_score'] / df[m_ind] * 40,
    0
)


In [48]:
# Usage & Pace Shares
# Minutes share and possessions share
df['min_share'] = np.where(
    df[m_team] > 0,
    df[m_ind] / (df[m_team] / df[gp_col]),
    0
)
# If you computed team possessions earlier, use that column name:
# df['poss_share'] = df['possessions'] / df['team_possessions']

In [49]:
# ── FINAL EXPORT CELL ──

# Base columns you already had before enrichment
out_cols_before = [
    'college','season',
    pn_col, p_col,
    gp_col, gs_col,
    m_ind, pts_ind, reb_tot, ast_ind, stl_ind, blk_ind, to_ind, 
    'pts_per40','reb_per40','ast_per40','stl_per40','blk_per40','tov_per40',
    'eFG_pct','TS_pct','USG_pct','ORB_pct','DRB_pct','AST_pct','AST_per_TO'
]

# Columns from enrichment block 
enrich1 = [
    '2pt_pct','FTr','PPP','possessions',
    'scoring_pts_per100','ast_per100','rebounds_tot_per100',
    'stl_per100','blk_per100','to_per100'
]
enrich2 = [
    # per-game stats
    'mins_per_game','pts_per_game','ast_per_game',
    'reb_per_game','stl_per_game','blk_per_game','to_per_game',
    # share metrics
    'scoring_pts_share','ast_share','rebounds_tot_share',
    'stl_share','blk_share','to_share',
    # efficiency differential
    'team_TS_pct','TS_diff',
    # assist/turnover ratios
    'ast_per_fgm','tov_rate',
    # game score
    'game_score','game_score_per40',
    # usage & pace shares
    'min_share'  # add 'poss_share' here if you calculated team possessions
]

# Combine them all
out_cols = out_cols_before + enrich1 + enrich2

# (5) Optional: drop any raw columns you no longer need
# drop_cols = ['fg_fga_ind','fg_fgm_ind','3pt_3pta_ind','3pt_ind','ft_fta_ind','ft_ftm_ind']
# df.drop(columns=[c for c in drop_cols if c in df], inplace=True)

# Export to CSV
df.to_csv('player_features.csv', columns=out_cols, index=False)

# Sanity check
print(f"Written player_features.csv with shape {df[out_cols].shape}")
print("Columns exported:")
for col in out_cols:
    print("  -", col)


Written player_features.csv with shape (2065, 56)
Columns exported:
  - college
  - season
  - player_number_ind
  - player_ind
  - gp_ind
  - gs_ind
  - minutes_tot_ind
  - scoring_pts_ind
  - rebounds_tot_ind
  - ast_ind
  - stl_ind
  - blk_ind
  - to_ind
  - pts_per40
  - reb_per40
  - ast_per40
  - stl_per40
  - blk_per40
  - tov_per40
  - eFG_pct
  - TS_pct
  - USG_pct
  - ORB_pct
  - DRB_pct
  - AST_pct
  - AST_per_TO
  - 2pt_pct
  - FTr
  - PPP
  - possessions
  - scoring_pts_per100
  - ast_per100
  - rebounds_tot_per100
  - stl_per100
  - blk_per100
  - to_per100
  - mins_per_game
  - pts_per_game
  - ast_per_game
  - reb_per_game
  - stl_per_game
  - blk_per_game
  - to_per_game
  - scoring_pts_share
  - ast_share
  - rebounds_tot_share
  - stl_share
  - blk_share
  - to_share
  - team_TS_pct
  - TS_diff
  - ast_per_fgm
  - tov_rate
  - game_score
  - game_score_per40
  - min_share


In [50]:
df = pd.read_csv('player_features.csv')
print(df.shape)
df.head()

# Numeric overview
display(df.describe().T.assign(missing=lambda d: df.isna().sum()))

# Count of seasons & colleges
print("Seasons:", df['season'].nunique())
print("Colleges:", df['college'].nunique())

(2065, 56)


,count,mean,std,min,25%,50%,75%,max,missing
player_number_ind,2065.0,17.040678,12.395348,0.000000,5.000000,15.000000,24.000000,55.000000,0
gp_ind,2065.0,20.311864,10.980011,1.000000,11.000000,25.000000,30.000000,36.000000,0
gs_ind,2065.0,10.182082,11.746382,0.000000,0.000000,3.000000,22.000000,35.000000,0
minutes_tot_ind,2065.0,410.265860,345.279436,0.000000,71.000000,344.000000,709.000000,1225.000000,0
scoring_pts_ind,2065.0,123.766102,131.299955,0.000000,14.000000,77.000000,202.000000,745.000000,0
rebounds_tot_ind,2065.0,64.047458,64.803076,0.000000,9.000000,48.000000,99.000000,419.000000,0
ast_ind,2065.0,26.008717,31.970567,0.000000,2.000000,15.000000,39.000000,232.000000,0
stl_ind,2065.0,16.014044,17.854258,0.000000,2.000000,11.000000,25.000000,135.000000,0
blk_ind,2065.0,6.104600,10.514603,0.000000,0.000000,2.000000,7.000000,121.000000,0
to_ind,2065.0,32.441162,30.135780,0.000000,6.000000,25.000000,52.000000,157.000000,0


Seasons: 21
Colleges: 13


In [51]:
# List columns with missing values
missing_columns = df.columns[df.isna().any()].tolist()
print("Columns with missing values:", missing_columns)


Columns with missing values: ['AST_per_TO', '2pt_pct', 'PPP', 'possessions', 'scoring_pts_per100', 'ast_per100', 'rebounds_tot_per100', 'stl_per100', 'blk_per100', 'to_per100']


In [52]:
# Load your dataset

for col in missing_columns:
    print(f"Column: {col}")
    missing = df[df[col].isna()]
    missing_players = missing[['player_ind', 'season']]
    print(missing_players)
    print("\n")

Column: AST_per_TO
                  player_ind   season
26              Emma Bedeker  2016-17
57               Jessica Lee  2021-22
70            Diamond Shavis  2018-19
82         Alexis Rosenfeld   2024-25
83       Caroline de Klauman  2024-25
...                      ...      ...
2004             Emily Brown  2012-13
2005       Bridget Whitfield  2012-13
2011          Sherika Salmon  2012-13
2033  Cristina Rojas Acevedo  2022-23
2064       Cinnamon Dockery   2020-21

[144 rows x 2 columns]


Column: 2pt_pct
                  player_ind   season
15              Emma Bedeker  2017-18
26              Emma Bedeker  2016-17
57               Jessica Lee  2021-22
83       Caroline de Klauman  2024-25
107             Becca Wilson  2023-24
...                      ...      ...
2017         Teresa Corchado  2012-13
2019          Lovisa Hagberg  2012-13
2020        Neechelle Ingram  2012-13
2034  Cristina Rojas Acevedo  2022-23
2064       Cinnamon Dockery   2020-21

[72 rows x 2 columns]


Co

In [53]:
for col in missing_columns:
    # Convert any infinities to NaN
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

    # Fill NaNs with 0 and assign back
    df[col] = df[col].fillna(0)

    # Check for any remaining missing values
    missing_counts = df.isna().sum()
    print("Columns with missing values:\n", missing_counts[missing_counts > 0])

# Confirm no missing values remain
display(df.describe().T.assign(missing=lambda d: df.isna().sum()))

Columns with missing values:
 2pt_pct                72
PPP                    70
possessions            35
scoring_pts_per100     35
ast_per100             35
rebounds_tot_per100    35
stl_per100             35
blk_per100             35
to_per100              35
dtype: int64
Columns with missing values:
 PPP                    70
possessions            35
scoring_pts_per100     35
ast_per100             35
rebounds_tot_per100    35
stl_per100             35
blk_per100             35
to_per100              35
dtype: int64
Columns with missing values:
 possessions            35
scoring_pts_per100     35
ast_per100             35
rebounds_tot_per100    35
stl_per100             35
blk_per100             35
to_per100              35
dtype: int64
Columns with missing values:
 scoring_pts_per100     35
ast_per100             35
rebounds_tot_per100    35
stl_per100             35
blk_per100             35
to_per100              35
dtype: int64
Columns with missing values:
 ast_per100        

,count,mean,std,min,25%,50%,75%,max,missing
player_number_ind,2065.0,17.040678,12.395348,0.000000,5.000000,15.000000,24.000000,55.000000,0
gp_ind,2065.0,20.311864,10.980011,1.000000,11.000000,25.000000,30.000000,36.000000,0
gs_ind,2065.0,10.182082,11.746382,0.000000,0.000000,3.000000,22.000000,35.000000,0
minutes_tot_ind,2065.0,410.265860,345.279436,0.000000,71.000000,344.000000,709.000000,1225.000000,0
scoring_pts_ind,2065.0,123.766102,131.299955,0.000000,14.000000,77.000000,202.000000,745.000000,0
rebounds_tot_ind,2065.0,64.047458,64.803076,0.000000,9.000000,48.000000,99.000000,419.000000,0
ast_ind,2065.0,26.008717,31.970567,0.000000,2.000000,15.000000,39.000000,232.000000,0
stl_ind,2065.0,16.014044,17.854258,0.000000,2.000000,11.000000,25.000000,135.000000,0
blk_ind,2065.0,6.104600,10.514603,0.000000,0.000000,2.000000,7.000000,121.000000,0
to_ind,2065.0,32.441162,30.135780,0.000000,6.000000,25.000000,52.000000,157.000000,0


In [54]:
#export cleaned dataframe
df.to_csv('player_features_cleaned.csv', index=False)
print("player_features_cleaned.csv saved, shape:", df.shape)

player_features_cleaned.csv saved, shape: (2065, 56)
